<a href="https://colab.research.google.com/github/Tiru-Kaggundi/Crypto_learning/blob/main/crypto2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Digital Signatures: ECDSA (secp256k1) — Colab-ready demo
# --------------------------------------------------------
# What you'll see:
# 1) Generate keypair (private/public)
# 2) Sign a message (hash-then-sign with SHA-256)
# 3) Verify the signature
# 4) Tampering detection (fails to verify)
# 5) Different key cannot verify (authenticity)
# 6) BONUS: Nonce-reuse vulnerability — recover the private key if 'k' is reused!

!pip -q install ecdsa

import os, hashlib, binascii
from ecdsa import SigningKey, VerifyingKey, SECP256k1, util
from ecdsa.util import sigencode_der, sigdecode_der



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 5.1 MB/s eta 0:00:00


In [ ]:

# --- Helpers ---
def sha256_bytes(m: bytes) -> bytes:
    return hashlib.sha256(m).digest()

def to_hex(b: bytes) -> str:
    return binascii.hexlify(b).decode()

# --- 1) Generate a keypair (secp256k1) ---
sk = SigningKey.generate(curve=SECP256k1)     # private key
vk = sk.get_verifying_key()                   # public key

print("=== Keypair (secp256k1) ===")
print("Private key (32B):", to_hex(sk.to_string()))
print("Public key  (64B, uncompressed XY):", to_hex(vk.to_string()))
print()


In [ ]:

# --- 2) Sign a message (hash then sign) ---
message = b"Pay 100 units to Alice"
msg_hash = sha256_bytes(message)
signature_der = sk.sign(msg_hash, sigencode=sigencode_der, hashfunc=hashlib.sha256)

print("=== Sign Message ===")
print("Message:", message.decode())
print("SHA-256(message):", to_hex(msg_hash))
print("Signature (DER):", to_hex(signature_der))
print()


In [ ]:

# --- 3) Verify the signature ---
try:
    ok = vk.verify(signature_der, msg_hash, sigdecode=sigdecode_der, hashfunc=hashlib.sha256)
    print("=== Verify ===")
    print("Valid signature?", ok)
except Exception as e:
    print("Verification error:", e)
print()


In [ ]:

# --- 4) Tampering test: change the message, reuse same signature -> verification should fail ---
tampered = b"Pay 1000 units to Alice"   # modified
tampered_hash = sha256_bytes(tampered)
try:
    ok_tampered = vk.verify(signature_der, tampered_hash, sigdecode=sigdecode_der, hashfunc=hashlib.sha256)
    print("=== Tampering Test ===")
    print("Valid on tampered message?", ok_tampered)
except Exception:
    print("=== Tampering Test ===")
    print("Valid on tampered message?", False)
print()


In [ ]:

# --- 5) Different key cannot verify (authenticity / non-repudiation idea) ---
sk_other = SigningKey.generate(curve=SECP256k1)
vk_other = sk_other.get_verifying_key()
try:
    ok_other = vk_other.verify(signature_der, msg_hash, sigdecode=sigdecode_der, hashfunc=hashlib.sha256)
    print("=== Wrong Key Test ===")
    print("Other key verifies?", ok_other)
except Exception:
    print("=== Wrong Key Test ===")
    print("Other key verifies?", False)
print()


In [ ]:

# --- 6) BONUS: Nonce (k) reuse attack demo (educational) ---
# ECDSA signs with a random nonce 'k'. If the signer mistakenly reuses the same k for two different messages,
# the private